In [1]:
NAME = 'Jason Ehlenberger'
COLLABORATORS = 'N/A'

In [2]:
import os
import zipfile as zp
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
import matplotlib.pyplot as plt
import rasterio
from rasterio.plot import show
from rasterio.plot import show_hist
from rasterio.warp import calculate_default_transform, reproject, Resampling
from rasterstats import zonal_stats

In [3]:
# Path to the directory
directory = 'Data\Assignment2'

# Check if the directory exists
if os.path.isdir(directory):
    src_dir = directory
else:
    src_dir = None

print(src_dir)

Data\Assignment2


In [4]:
def data_path(directory, filename):
    file_path = os.path.join(directory, filename)
    return file_path

In [5]:
# Use the new function to set the path
shpfn = 'counties_subset_Albers4.shp'
data = data_path(src_dir, shpfn)

rstfn = 'crop_cover_2019.tif'
raster = rasterio.open(data_path(src_dir, rstfn))

type(raster)

rasterio.io.DatasetReader

In [6]:
raster.crs

CRS.from_epsg(5070)

In [7]:
with rasterio.open(data_path(src_dir, 'crop_cover_2019.tif')) as raster_crop:
    crop_val = raster_crop.read()
crop_val

array([[[176, 176, 176, ...,   1,   5,   5],
        [176, 176, 176, ..., 176,  26, 141],
        [176, 176, 176, ...,   1, 141,   1],
        ...,
        [176, 121, 121, ..., 141, 121, 141],
        [121, 121, 176, ..., 176, 121, 141],
        [176, 176, 176, ..., 141, 121, 141]]], dtype=uint8)

In [8]:
raster.bounds

BoundingBox(left=-30012.21913293552, bottom=1544807.0280387579, right=154937.7808670645, top=1673717.0280387579)

In [9]:
# Raster units & Resolution
print(f'The raster units are {raster.crs.linear_units}s and the resolution is {raster.res}.')

The raster units are metres and the resolution is (30.0, 30.0).


In [10]:
dst_crs = 'EPSG:9712'

In [11]:
# step 1: Calculate transform array and shape of reprojected raster 
with rasterio.open(data_path(src_dir, 'crop_cover_2019.tif')) as src:
    transform, width, height = calculate_default_transform(
        src.crs, dst_crs, src.width, src.height, *src.bounds)

In [12]:
    # step 2: We copy metadata from source dataset to dictionary object `kwargs`
    kwargs = src.meta.copy()
    # step 3: we update information in kwargs object with destination crs 
    kwargs.update({
    'crs': dst_crs,
    'transform': transform, 
    'width': width, 
    'height': height
    })

In [ ]:
 # step 4: for each band (i), we reproject the calculated parameters froms source to destination      
    with rasterio.open(data_path(src_dir,'crop_2018_subset2_pr.tif', 'w', **kwargs)) as dst:
        for i in range(1, src.count +1):
            reproject(
                source = rasterio.band(src, i), 
                destination = rasterio.band(dst, i), 
                src_transform = src.transform, 
                src_crs = src.crs, 
                dst_transform = transform, 
                dst_crs = dst_crs, 
                resampling = Resampling.nearest)
    # make sure to close the raster file
        dst.close()